In [1]:
import os

In [2]:
!cd

c:\Users\Sahil\OneDrive\AI_ADV\ChickenDisease\Chicken-Disease-Classification-using-Fecal-Image\research


In [3]:
os.chdir("../")

In [4]:
!cd

c:\Users\Sahil\OneDrive\AI_ADV\ChickenDisease\Chicken-Disease-Classification-using-Fecal-Image


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epoch=params.EPOCHS,  
            params_batch_size=params.BATCH_SIZE,  
            params_is_augmentation=params.AUGMENTATION,  
            params_image_size=params.IMAGE_SIZE  
)



        return training_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import time
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

[2025-03-08 22:14:38,573]: WARNING: module_wrapper: From C:\Users\Sahil\AppData\Local\Temp\ipykernel_7596\271509718.py:6: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
]


In [16]:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Set appropriate learning rate
        loss='categorical_crossentropy',  # Use the correct loss function
        metrics=['accuracy']
    )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        tf.config.run_functions_eagerly(True)
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [19]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-03-08 22:27:15,479]: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-08 22:27:15,496]: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-08 22:27:15,499]: INFO: common: created directory at: artifacts]
[2025-03-08 22:27:15,502]: INFO: common: created directory at: artifacts\training]
[2025-03-08 22:27:15,829]: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 134s 7s/step - accuracy: 0.6486 - loss: 0.9399 - val_accuracy: 0.7656 - val_loss: 0.6350
Epoch 2/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 416ms/step - accuracy: 0.8125 - loss: 0.2983 - val_accuracy: 0.7500 - val_loss: 0.6723
Epoch 3/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 130s 7s/step - accuracy: 0.9123 - loss: 0.2931 - val_accuracy: 0.9219 - val_loss: 0.3022
Epo